In [1]:
# Top 50 US cities by population with their center coordinates (lat, lon)
TOP_50_CITIES = {
    "New York": (40.7128, -74.0060),
    "Los Angeles": (34.0522, -118.2437),
    "Chicago": (41.8781, -87.6298),
    "Houston": (29.7604, -95.3698),
    "Phoenix": (33.4484, -112.0740),
    "Philadelphia": (39.9526, -75.1652),
    "San Antonio": (29.4241, -98.4936),
    "San Diego": (32.7157, -117.1611),
    "Dallas": (32.7767, -96.7970),
    "San Jose": (37.3382, -121.8863),
    "Austin": (30.2672, -97.7431),
    "Jacksonville": (30.3322, -81.6557),
    "Fort Worth": (32.7555, -97.3308),
    "Columbus": (39.9612, -82.9988),
    "Charlotte": (35.2271, -80.8431),
    "San Francisco": (37.7749, -122.4194),
    "Indianapolis": (39.7684, -86.1581),
    "Seattle": (47.6062, -122.3321),
    "Denver": (39.7392, -104.9903),
    "Washington DC": (38.9072, -77.0369),
    "Boston": (42.3601, -71.0589),
    "El Paso": (31.7619, -106.4850),
    "Nashville": (36.1627, -86.7816),
    "Detroit": (42.3314, -83.0458),
    "Oklahoma City": (35.4676, -97.5164),
    "Portland": (45.5152, -122.6784),
    "Las Vegas": (36.1699, -115.1398),
    "Memphis": (35.1495, -90.0490),
    "Louisville": (38.2527, -85.7585),
    "Baltimore": (39.2904, -76.6122),
    "Milwaukee": (43.0389, -87.9065),
    "Albuquerque": (35.0844, -106.6504),
    "Tucson": (32.2226, -110.9747),
    "Fresno": (36.7378, -119.7871),
    "Mesa": (33.4152, -111.8315),
    "Sacramento": (38.5816, -121.4944),
    "Kansas City": (39.0997, -94.5786),
    "Colorado Springs": (38.8339, -104.8214),
    "Omaha": (41.2565, -95.9345),
    "Raleigh": (35.7796, -78.6382),
    "Miami": (25.7617, -80.1918),
    "Long Beach": (33.7701, -118.1937),
    "Virginia Beach": (36.8529, -75.9780),
    "Oakland": (37.8044, -122.2712),
    "Minneapolis": (44.9778, -93.2650),
    "Tulsa": (36.1540, -95.9928),
    "Tampa": (27.9506, -82.4572),
    "Arlington": (32.7357, -97.1081),
    "New Orleans": (29.9511, -90.0715)
}

In [2]:
# Import relevant components from Sionna RT
import matplotlib.pyplot as plt
import numpy as np
import mitsuba as mi
import warnings
import sys
import os

from sionna.rt import load_scene, Transmitter, Receiver, Transmitter, Camera, PathSolver, RadioMapSolver
from sionna.rt import AntennaArray, PlanarArray, SceneObject, ITURadioMaterial
from sionna.rt.antenna_pattern import antenna_pattern_registry

# Add the src directory to the Python path
sys.path.append(os.path.abspath('../src'))

# Building placement code
from scene_parser import extract_building_info
from tx_placement import TxPlacement
from boresight_pathsolver import create_zone_mask, optimize_boresight_pathsolver
from angle_utils import azimuth_elevation_to_yaw_pitch
from zone_validator import find_valid_zone
from boresight_pathsolver import compare_boresight_performance

# Running the optimizations on 50 scenes
# These scenes make up the 50 most populous cities in the United States
# Experiment Description (Per Scene):
# Map: 1km x 1 km
# Target: 250m x 250m square zones
# Scenarios: Zone placed 200-400m from TX to ensure multipath propagation
# Frequency: 3.67 GHz
# Tx Placement: Most central building with a 5.0 m offset in Z
# UE Z-Height: 1.5 m
# Initial Orientation: Naive Baseline -> Centroid
# Loss function is single objective: Raise Geometric Mean
# Analysis: RadioMapSolver -> Zone Power per 1x1 meter cell (BEFORE and AFTER optimization)
# Choice of LDS: Sobol, Halton, Latin
# Choice of Sampling Method: Rejection, CDT + Turk's 
# Zone Validation: Ensures high-stakes scenarios (-140 <= p10 <= -80 dBm, p90 >= -130 dBm, range >= 15 dB)

jit_find_library(): Unable to load "/usr/lib/x86_64-linux-gnu/libLLVM-20.so": /home/tingjunlab/anaconda3/lib/python3.13/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/lib/x86_64-linux-gnu/libLLVM-20.so)!
jitc_llvm_init(): LLVM API initialization failed ..


In [3]:
# Get list of scene subdirectories and sort them alphabetically
parent_folder = "../scene/scenes"
scene_dirs = sorted([d for d in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, d))])

# Dictionary to store all results keyed by scene name
results = {}

# RadioMapSolver()
rm_solver = RadioMapSolver()

# Zone validation thresholds for high-stakes scenarios
validation_thresholds = {
    'p10_min_dbm': -200.0,              # Reject zones with 10th percentile < -200 dBm (too weak/dead)
    'p10_max_dbm': -90.0,               # Reject zones with 10th percentile > -80 dBm (too strong)
    'p90_min_dbm': -105.0,              # Reject zones with 90th percentile < -130 dBm (too weak)
    'min_percentile_range_db': 15.0,    # Require at least 15 dB range between p90 and p10
    'median_max_dbm': -60.0             # I don't want a median that's already too healthy
}

for i, scene_name in enumerate(scene_dirs[:3]):    
    print(f"Scene {i+1}/{len(scene_dirs)}: {scene_name}")
    # Build path to scene XML
    scene_xml_path = os.path.join(parent_folder, scene_name, "scene.xml")
    # Load scene
    scene = load_scene(scene_xml_path)

    # Set a dummy frequency (start with the highest value supported by map materials to ensure propagation is retained)
    scene.frequency = 3.7e9 #9.99 GHz

    # gNB antenna: 3GPP TR 38.901 pattern (AIRSTRAN D 2200)
    gnb_pattern_factory = antenna_pattern_registry.get("tr38901")
    gnb_pattern = gnb_pattern_factory(polarization="V")

    # UE antenna: Isotropic pattern (typical for mobile devices)
    # This will be required for matching the calculations of the RadioMapSolver()
    ue_pattern_factory = antenna_pattern_registry.get("iso")
    # Polarization should also match the transmitter
    ue_pattern = ue_pattern_factory(polarization="V")

    # SISO: Single antenna element at origin [0, 0, 0] for both TX and RX
    single_element = np.array([[0.0, 0.0, 0.0]])  # Shape: (1, 3)

    # Configure antenna arrays
    scene.tx_array = AntennaArray(
        antenna_pattern=gnb_pattern,
        normalized_positions=single_element.T  # Shape: (3, 1)
    )

    scene.rx_array = AntennaArray(
        antenna_pattern=ue_pattern,
        normalized_positions=single_element.T  # Shape: (3, 1)
    )

    # Disable scattering for basic simulation
    for radio_material in scene.radio_materials.values():
        radio_material.scattering_coefficient = 0.0

    # Select building and establish the transmitter
    building_info = extract_building_info(scene_xml_path, verbose=False)

    # Find the most central building (closest to 0,0)
    min_distance = float('inf')
    selected_building_id = None
    
    for building_id, info in building_info.items():
        x_center, y_center = info['center']
        # Calculate Euclidean distance from (0, 0)
        distance = np.sqrt(x_center**2 + y_center**2)
        if distance < min_distance:
            min_distance = distance
            selected_building_id = building_id
    
    print(f"Selected most central building: {selected_building_id} (distance from origin: {min_distance:.2f}m)")

    # TxPlacement will create the transmitter if it doesn't exist and place it on the building
    # Correct parameter order: (scene, tx_name, scene_xml_path, building_id, offset)
    tx_placer = TxPlacement(scene, "gnb", scene_xml_path, selected_building_id, offset=5.0)
    tx_placer.set_rooftop_center()

    # Get reference to the transmitter (already added to scene by TxPlacement)
    tx = tx_placer.tx
    # Convert to flat numpy array instead of nested list
    gnb_position = tx.position.numpy().flatten().tolist()
    
    print(f"gNB placed on building {selected_building_id} at position: {gnb_position}")  

    # Map Data
    # Define the map configuration
    map_config = {
        'center': [0.0, 0.0, 0.0],
        'size': [1000, 1000],
        'cell_size': (1.0, 1.0),
        'ground_height': 0.0,
    }   

    # Find a valid zone using automatic search with validation
    zone_params_template = {
        'width': 250.0,
        'height': 250.0
    }
    
    zone_mask, zone_stats, zone_center, validation_stats, attempts = find_valid_zone(
        scene=scene,
        tx_name="gnb",
        tx_position=gnb_position,
        map_config=map_config,
        scene_xml_path=scene_xml_path,
        zone_params_template=zone_params_template,
        min_distance=50.0,
        max_distance=300.0,
        max_attempts=200,
        validation_kwargs=validation_thresholds,
        verbose=True
    )
    
    # Check if valid zone was found
    if zone_mask is None:
        print(f"✗ Could not find valid zone for {scene_name} after {attempts} attempts - skipping scene\n")
        results[scene_name] = {
            'status': 'failed',
            'reason': 'No valid zone found',
            'attempts': attempts
        }
        print("="*80 + "\n")
        continue
    
    zone_center_x, zone_center_y = zone_center
    zone_params = zone_stats['zone_params']
    
    print(f"✓ Found valid zone after {attempts} attempt(s)")
    print(f"  Zone center: ({zone_center_x:.1f}, {zone_center_y:.1f})")
    print(f"  P10: {validation_stats['p10_power_dbm']:.1f} dBm, P90: {validation_stats['p90_power_dbm']:.1f} dBm")
    print(f"  Percentile range: {validation_stats['percentile_range_db']:.1f} dB")
    
    # Calculate zone distance and angle from TX for logging
    zone_distance_from_tx = np.sqrt((zone_center_x - gnb_position[0])**2 + (zone_center_y - gnb_position[1])**2)
    zone_angle_from_tx = np.arctan2(zone_center_y - gnb_position[1], zone_center_x - gnb_position[0])

    # Reset scene frequency
    scene.frequency = 3.7e9
    
    # Run optimization!
    best_angles, loss_hist, angle_hist, grad_hist, cov_stats, initial_angles = optimize_boresight_pathsolver(
        scene=scene,
        tx_name="gnb",
        map_config=map_config,
        scene_xml_path=scene_xml_path,
        zone_mask=zone_mask,
        zone_params=zone_params,
        zone_stats=zone_stats,
        num_sample_points=50,
        learning_rate=2.0,
        num_iterations=100,
        verbose=True,
        lds="Latin",
        sampler="Rejection"
    )
    
    print(f"\nOptimization complete!")
    print(f"  Initial angles: Azimuth={initial_angles[0]:.1f}°, Elevation={initial_angles[1]:.1f}°")
    print(f"  Best angles:    Azimuth={best_angles[0]:.1f}°, Elevation={best_angles[1]:.1f}°")

    # ===== EVALUATION WITH INITIAL ANGLES (BASELINE) =====
    print(f"\nEvaluating with INITIAL angles...")
    # Set transmitter orientation to initial angles
    yaw_initial, pitch_initial = azimuth_elevation_to_yaw_pitch(initial_angles[0], initial_angles[1])
    tx.orientation = mi.Point3f(float(yaw_initial), float(pitch_initial), 0.0)
    
    # Generate radio map with initial orientation
    rm_initial = rm_solver(
        scene,
        max_depth=5,
        samples_per_tx=int(6e8),
        cell_size=map_config['cell_size'],
        center=map_config['center'],
        orientation=[0, 0, 0],
        size=map_config['size'],
        los=True,
        specular_reflection=True,
        diffuse_reflection=True,
        diffraction=True,
        edge_diffraction=True,
        refraction=False,
        stop_threshold=None,
    )
    
    # Extract zone power for initial configuration
    rss_watts_initial = rm_initial.rss.numpy()[0, :, :]
    signal_strength_dBm_initial = 10.0 * np.log10(rss_watts_initial + 1e-30) + 30.0
    zone_power_initial = signal_strength_dBm_initial[zone_mask == 1.0]
    
    print(f"  Initial mean power: {np.mean(zone_power_initial):.2f} dBm")

    # ===== EVALUATION WITH OPTIMIZED ANGLES =====
    print(f"Evaluating with OPTIMIZED angles...")
    # Set transmitter orientation to best angles
    best_azimuth, best_elevation = azimuth_elevation_to_yaw_pitch(best_angles[0], best_angles[1])
    tx.orientation = mi.Point3f(float(best_azimuth), float(best_elevation), 0.0)
    
    # Generate radio map with optimized orientation
    rm_optimized = rm_solver(
        scene,
        max_depth=5,
        samples_per_tx=int(6e8),
        cell_size=map_config['cell_size'],
        center=map_config['center'],
        orientation=[0, 0, 0],
        size=map_config['size'],
        los=True,
        specular_reflection=True,
        diffuse_reflection=True,
        diffraction=True,
        edge_diffraction=True,
        refraction=False,
        stop_threshold=None,
    )
    
    # Extract zone power for optimized configuration
    rss_watts_optimized = rm_optimized.rss.numpy()[0, :, :]
    signal_strength_dBm_optimized = 10.0 * np.log10(rss_watts_optimized + 1e-30) + 30.0
    zone_power = signal_strength_dBm_optimized[zone_mask == 1.0]

    print("Coverage Analysis:")
    print(f"  Mean power in zone: {np.mean(zone_power):.2f} dBm")
    print(f"  Median power in zone: {np.median(zone_power):.2f} dBm")
    print(f"  Min power in zone: {np.min(zone_power):.2f} dBm")
    print(f"  Max power in zone: {np.max(zone_power):.2f} dBm")
    print(f"  Std dev in zone: {np.std(zone_power):.2f} dB")
    print()

    # ============================================
    # Optimization Diagnostics - UPDATED for Angles
    # ============================================
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Loss history
    axes[0, 0].plot(loss_hist, 'b-', linewidth=2)
    axes[0, 0].set_title('Loss History')
    axes[0, 0].set_xlabel('Iteration')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True, alpha=0.3)

    # Gradient norms
    axes[0, 1].plot(grad_hist, 'r-', linewidth=2)
    axes[0, 1].set_title('Gradient Norm History')
    axes[0, 1].set_xlabel('Iteration')
    axes[0, 1].set_ylabel('Gradient Norm')
    axes[0, 1].set_yscale('log')
    axes[0, 1].grid(True, alpha=0.3)

    # Angle trajectory - Azimuth
    angle_arr = np.array(angle_hist)
    axes[1, 0].plot(angle_arr[:, 0], 'g-', linewidth=2, label='Azimuth')
    axes[1, 0].axhline(y=best_azimuth, color='b', linestyle='--', label=f'Final: {best_azimuth:.1f}°')
    axes[1, 0].set_title('Azimuth Angle Optimization')
    axes[1, 0].set_xlabel('Iteration')
    axes[1, 0].set_ylabel('Azimuth (degrees)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Angle trajectory - Elevation
    axes[1, 1].plot(angle_arr[:, 1], 'm-', linewidth=2, label='Elevation')
    axes[1, 1].axhline(y=best_elevation, color='b', linestyle='--', label=f'Final: {best_elevation:.1f}°')
    axes[1, 1].set_title('Elevation Angle Optimization')
    axes[1, 1].set_xlabel('Iteration')
    axes[1, 1].set_ylabel('Elevation (degrees)')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Compare performance using angles
    fig, comparison_stats = compare_boresight_performance(
        scene=scene,
        tx_name="gnb",
        map_config=map_config,
        zone_mask=zone_mask,
        naive_angles=initial_angles,
        optimized_angles=best_angles,
        title="Boresight Optimization Results (Angle-Based)"
    )

    plt.show()

    # Save all relevant data to dictionary
    results[scene_name] = {
        'status': 'success',
        'scene_xml_path': scene_xml_path,
        'map_config': map_config,
        'initial_angles': initial_angles,
        'best_angles': best_angles,
        'loss_hist': loss_hist,
        'angle_hist': angle_hist,
        'grad_hist': grad_hist,
        'cov_stats': cov_stats,
        'tx_building_id': selected_building_id,
        'tx_position': gnb_position,
        'zone_center': [zone_center_x, zone_center_y],
        'zone_distance_from_tx': zone_distance_from_tx,
        'zone_angle_from_tx': zone_angle_from_tx,
        'zone_attempts': attempts,
        'validation_stats': validation_stats,
        'zone_params': zone_params,
        'zone_stats': zone_stats,
        'zone_power_initial': zone_power_initial,
        'zone_power_optimized': zone_power
    }

    print(f"\nResults saved for {scene_name}")
    print("="*80 + "\n")

In [4]:
# Update the material definitions to use the updated ground materials
import sys
import sionna.rt

g2sm = "/home/tingjunlab/Development/geo2sigmap"
sys.path.append(g2sm)
from geo2sigmap.materials.csv_interpolator import create_custom_material

# New material definitions
new_wet_ground = create_custom_material(
    "new_wet_ground",
    "/home/tingjunlab/Development/geo2sigmap/research/data/ITU-R_P.527-3_extracted_data/B_wet_ground_per.csv",
    "/home/tingjunlab/Development/geo2sigmap/research/data/ITU-R_P.527-3_extracted_data/B_wet_ground_con.csv",
)
new_medium_dry_ground = create_custom_material(
    "new_medium_dry_ground",
    "/home/tingjunlab/Development/geo2sigmap/research/data/ITU-R_P.527-3_extracted_data/D_medium_dry_ground_per.csv",
    "/home/tingjunlab/Development/geo2sigmap/research/data/ITU-R_P.527-3_extracted_data/D_medium_dry_ground_con.csv",
)
new_very_dry_ground = create_custom_material(
    "new_very_dry_ground",
    "/home/tingjunlab/Development/geo2sigmap/research/data/ITU-R_P.527-3_extracted_data/E_very_dry_ground_per.csv",
    "/home/tingjunlab/Development/geo2sigmap/research/data/ITU-R_P.527-3_extracted_data/E_very_dry_ground_con.csv",
)

Creating a custom material
Creating a custom material
Creating a custom material


In [5]:
print(f"Testing {len(scene_dirs)} scenes")
print(f"Validation thresholds: {validation_thresholds}\n")

for i, scene_name in enumerate(scene_dirs[:99]):    
    print(f"Scene {i+1}/{len(scene_dirs)}: {scene_name}")
    # Build path to scene XML
    scene_xml_path = os.path.join(parent_folder, scene_name, "scene.xml")
    # Load scene
    scene = load_scene(scene_xml_path)

    # Replace ground material with custom material that supports FR3
    ground = scene.get("ground")
    ground.radio_material = new_wet_ground

    # Replace ALL ITU materials in the scene that might not support 12 GHz
    from sionna.rt.radio_materials.itu_material import ITURadioMaterial
    
    for mat_name, mat_obj in list(scene._radio_materials.items()):
        if isinstance(mat_obj, ITURadioMaterial):
            # Replace with appropriate custom material based on name
            if "wet_ground" in mat_name.lower() or "ground" in mat_name.lower():
                print(f"  → Replacing '{mat_name}' with custom wet_ground material")
                scene._radio_materials[mat_name] = new_wet_ground
            elif "dry" in mat_name.lower():
                print(f"  → Replacing '{mat_name}' with custom very_dry_ground material")
                scene._radio_materials[mat_name] = new_very_dry_ground
            else:
                # Default replacement for other ITU materials
                print(f"  → Replacing '{mat_name}' with custom wet_ground material (default)")
                scene._radio_materials[mat_name] = new_wet_ground

    # Now it's safe to set the frequency - all materials support FR3
    scene.frequency = 12e9  # FR3 frequency
    print(f"✓ Frequency set to {scene.frequency/1e9} GHz")

    # gNB antenna: 3GPP TR 38.901 pattern (AIRSTRAN D 2200)
    gnb_pattern_factory = antenna_pattern_registry.get("tr38901")
    gnb_pattern = gnb_pattern_factory(polarization="V")

    # UE antenna: Isotropic pattern (typical for mobile devices)
    # This will be required for matching the calculations of the RadioMapSolver()
    ue_pattern_factory = antenna_pattern_registry.get("iso")
    # Polarization should also match the transmitter
    ue_pattern = ue_pattern_factory(polarization="V")

    # SISO: Single antenna element at origin [0, 0, 0] for both TX and RX
    single_element = np.array([[0.0, 0.0, 0.0]])  # Shape: (1, 3)

    # Configure antenna arrays
    scene.tx_array = AntennaArray(
        antenna_pattern=gnb_pattern,
        normalized_positions=single_element.T  # Shape: (3, 1)
    )

    scene.rx_array = AntennaArray(
        antenna_pattern=ue_pattern,
        normalized_positions=single_element.T  # Shape: (3, 1)
    )

    # Disable scattering for basic simulation
    for radio_material in scene._radio_materials.values():
        radio_material.scattering_coefficient = 0.0

    # Select building and establish the transmitter
    building_info = extract_building_info(scene_xml_path, verbose=False)

    # Find the most central building (closest to 0,0)
    min_distance = float('inf')
    selected_building_id = None
    
    for building_id, info in building_info.items():
        x_center, y_center = info['center']
        # Calculate Euclidean distance from (0, 0)
        distance = np.sqrt(x_center**2 + y_center**2)
        if distance < min_distance:
            min_distance = distance
            selected_building_id = building_id
    
    print(f"Selected most central building: {selected_building_id} (distance from origin: {min_distance:.2f}m)")

    # TxPlacement will create the transmitter if it doesn't exist and place it on the building
    # Correct parameter order: (scene, tx_name, scene_xml_path, building_id, offset)
    tx_placer = TxPlacement(scene, "gnb", scene_xml_path, selected_building_id, offset=5.0)
    tx_placer.set_rooftop_center()

    # Get reference to the transmitter (already added to scene by TxPlacement)
    tx = tx_placer.tx
    # Convert to flat numpy array instead of nested list
    gnb_position = tx.position.numpy().flatten().tolist()
    
    print(f"gNB placed on building {selected_building_id} at position: {gnb_position}")  

    # Map Data
    # Define the map configuration
    map_config = {
        'center': [0.0, 0.0, 0.0],
        'size': [1000, 1000],
        'cell_size': (1.0, 1.0),
        'ground_height': 0.0,
    }   

    # Find a valid zone using automatic search with validation
    zone_params_template = {
        'width': 250.0,
        'height': 250.0
    }
    
    zone_mask, zone_stats, zone_center, validation_stats, attempts = find_valid_zone(
        scene=scene,
        tx_name="gnb",
        tx_position=gnb_position,
        map_config=map_config,
        scene_xml_path=scene_xml_path,
        zone_params_template=zone_params_template,
        min_distance=50.0,
        max_distance=300.0,
        max_attempts=200,
        validation_kwargs=validation_thresholds,
        verbose=True
    )
    
    # Check if valid zone was found
    if zone_mask is None:
        print(f"✗ Could not find valid zone for {scene_name} after {attempts} attempts - skipping scene\n")
        results[scene_name] = {
            'status': 'failed',
            'reason': 'No valid zone found',
            'attempts': attempts
        }
        print("="*80 + "\n")
        continue
    
    zone_center_x, zone_center_y = zone_center
    zone_params = zone_stats['zone_params']
    
    print(f"✓ Found valid zone after {attempts} attempt(s)")
    print(f"  Zone center: ({zone_center_x:.1f}, {zone_center_y:.1f})")
    print(f"  P10: {validation_stats['p10_power_dbm']:.1f} dBm, P90: {validation_stats['p90_power_dbm']:.1f} dBm")
    print(f"  Percentile range: {validation_stats['percentile_range_db']:.1f} dB")
    
    # Calculate zone distance and angle from TX for logging
    zone_distance_from_tx = np.sqrt((zone_center_x - gnb_position[0])**2 + (zone_center_y - gnb_position[1])**2)
    zone_angle_from_tx = np.arctan2(zone_center_y - gnb_position[1], zone_center_x - gnb_position[0])

    # Simple test matrix used to evaluate the same scene with different Samplers, Frequencies and Sequences
    for sampler in ["Rejection", "CDT"]:
        for freq in [2.4e9, 5.2e9, 7.1e9, 9.7e9]:
            for lds in ["Sobol", "Halton", "Latin"]:

                # Reset scene frequency
                scene.frequency = freq
                
                # Run optimization!
                best_angles, loss_hist, angle_hist, grad_hist, cov_stats, initial_angles = optimize_boresight_pathsolver(
                    scene=scene,
                    tx_name="gnb",
                    map_config=map_config,
                    scene_xml_path=scene_xml_path,
                    zone_mask=zone_mask,
                    zone_params=zone_params,
                    zone_stats=zone_stats,
                    num_sample_points=50,
                    learning_rate=2.0,
                    num_iterations=100,
                    verbose=True,
                    lds=lds,
                    sampler=sampler
                )
                
                print(f"\nOptimization complete!")
                print(f"  Initial angles: Azimuth={initial_angles[0]:.1f}°, Elevation={initial_angles[1]:.1f}°")
                print(f"  Best angles:    Azimuth={best_angles[0]:.1f}°, Elevation={best_angles[1]:.1f}°")

                # ===== EVALUATION WITH INITIAL ANGLES (BASELINE) =====
                print(f"\nEvaluating with INITIAL angles...")
                # Set transmitter orientation to initial angles
                yaw_initial, pitch_initial = azimuth_elevation_to_yaw_pitch(initial_angles[0], initial_angles[1])
                tx.orientation = mi.Point3f(float(yaw_initial), float(pitch_initial), 0.0)
                
                # Generate radio map with initial orientation
                rm_initial = rm_solver(
                    scene,
                    max_depth=8,
                    samples_per_tx=int(6e8),
                    cell_size=map_config['cell_size'],
                    center=map_config['center'],
                    orientation=[0, 0, 0],
                    size=map_config['size'],
                    los=True,
                    specular_reflection=True,
                    diffuse_reflection=True,
                    diffraction=True,
                    edge_diffraction=True,
                    refraction=False,
                    stop_threshold=None,
                )
                
                # Extract zone power for initial configuration
                rss_watts_initial = rm_initial.rss.numpy()[0, :, :]
                signal_strength_dBm_initial = 10.0 * np.log10(rss_watts_initial + 1e-30) + 30.0
                zone_power_initial = signal_strength_dBm_initial[zone_mask == 1.0]
                
                print(f"  Initial mean power: {np.mean(zone_power_initial):.2f} dBm")

                # ===== EVALUATION WITH OPTIMIZED ANGLES =====
                print(f"Evaluating with OPTIMIZED angles...")
                # Set transmitter orientation to best angles
                best_azimuth, best_elevation = azimuth_elevation_to_yaw_pitch(best_angles[0], best_angles[1])
                tx.orientation = mi.Point3f(float(best_azimuth), float(best_elevation), 0.0)
                
                # Generate radio map with optimized orientation
                rm_optimized = rm_solver(
                    scene,
                    max_depth=5,
                    samples_per_tx=int(6e8),
                    cell_size=map_config['cell_size'],
                    center=map_config['center'],
                    orientation=[0, 0, 0],
                    size=map_config['size'],
                    los=True,
                    specular_reflection=True,
                    diffuse_reflection=True,
                    diffraction=True,
                    edge_diffraction=True,
                    refraction=False,
                    stop_threshold=None,
                )
                
                # Extract zone power for optimized configuration
                rss_watts_optimized = rm_optimized.rss.numpy()[0, :, :]
                signal_strength_dBm_optimized = 10.0 * np.log10(rss_watts_optimized + 1e-30) + 30.0
                zone_power = signal_strength_dBm_optimized[zone_mask == 1.0]

                print("Coverage Analysis:")
                print(f"  Mean power in zone: {np.mean(zone_power):.2f} dBm")
                print(f"  Median power in zone: {np.median(zone_power):.2f} dBm")
                print(f"  Min power in zone: {np.min(zone_power):.2f} dBm")
                print(f"  Max power in zone: {np.max(zone_power):.2f} dBm")
                print(f"  Std dev in zone: {np.std(zone_power):.2f} dB")
                print()

                # ============================================
                # Optimization Diagnostics - UPDATED for Angles
                # ============================================
                fig, axes = plt.subplots(2, 2, figsize=(16, 12))

                # Loss history
                axes[0, 0].plot(loss_hist, 'b-', linewidth=2)
                axes[0, 0].set_title('Loss History')
                axes[0, 0].set_xlabel('Iteration')
                axes[0, 0].set_ylabel('Loss')
                axes[0, 0].grid(True, alpha=0.3)

                # Gradient norms
                axes[0, 1].plot(grad_hist, 'r-', linewidth=2)
                axes[0, 1].set_title('Gradient Norm History')
                axes[0, 1].set_xlabel('Iteration')
                axes[0, 1].set_ylabel('Gradient Norm')
                axes[0, 1].set_yscale('log')
                axes[0, 1].grid(True, alpha=0.3)

                # Angle trajectory - Azimuth
                angle_arr = np.array(angle_hist)
                axes[1, 0].plot(angle_arr[:, 0], 'g-', linewidth=2, label='Azimuth')
                axes[1, 0].axhline(y=best_azimuth, color='b', linestyle='--', label=f'Final: {best_azimuth:.1f}°')
                axes[1, 0].set_title('Azimuth Angle Optimization')
                axes[1, 0].set_xlabel('Iteration')
                axes[1, 0].set_ylabel('Azimuth (degrees)')
                axes[1, 0].legend()
                axes[1, 0].grid(True, alpha=0.3)

                # Angle trajectory - Elevation
                axes[1, 1].plot(angle_arr[:, 1], 'm-', linewidth=2, label='Elevation')
                axes[1, 1].axhline(y=best_elevation, color='b', linestyle='--', label=f'Final: {best_elevation:.1f}°')
                axes[1, 1].set_title('Elevation Angle Optimization')
                axes[1, 1].set_xlabel('Iteration')
                axes[1, 1].set_ylabel('Elevation (degrees)')
                axes[1, 1].legend()
                axes[1, 1].grid(True, alpha=0.3)

                plt.tight_layout()
                plt.show()

                # Compare performance using angles
                fig, comparison_stats = compare_boresight_performance(
                    scene=scene,
                    tx_name="gnb",
                    map_config=map_config,
                    zone_mask=zone_mask,
                    naive_angles=initial_angles,
                    optimized_angles=best_angles,
                    title="Boresight Optimization Results (Angle-Based)"
                )

                plt.show()

                # Save all relevant data to dictionary
                results[scene_name + sampler + f"{freq}" + lds] = {
                    'status': 'success',
                    'scene_xml_path': scene_xml_path,
                    'map_config': map_config,
                    'initial_angles': initial_angles,
                    'best_angles': best_angles,
                    'loss_hist': loss_hist,
                    'angle_hist': angle_hist,
                    'grad_hist': grad_hist,
                    'cov_stats': cov_stats,
                    'tx_building_id': selected_building_id,
                    'tx_position': gnb_position,
                    'zone_center': [zone_center_x, zone_center_y],
                    'zone_distance_from_tx': zone_distance_from_tx,
                    'zone_angle_from_tx': zone_angle_from_tx,
                    'zone_attempts': attempts,
                    'validation_stats': validation_stats,
                    'zone_params': zone_params,
                    'zone_stats': zone_stats,
                    'zone_power_initial': zone_power_initial,
                    'zone_power_optimized': zone_power
                }

                print(f"\nResults saved for {scene_name}")
                print("="*80 + "\n")

Testing 51 scenes
Validation thresholds: {'p10_min_dbm': -200.0, 'p10_max_dbm': -90.0, 'p90_min_dbm': -105.0, 'min_percentile_range_db': 15.0, 'median_max_dbm': -60.0}

Scene 1/51: Duke
2026-02-10 17:45:13 WARN  [HDRFilm] Monochrome mode enabled, setting film output pixel format to 'luminance' (was rgb).
  → Replacing 'itu_concrete' with custom wet_ground material (default)
  → Replacing 'itu_metal' with custom wet_ground material (default)
  → Replacing 'itu_wet_ground' with custom wet_ground material
✓ Frequency set to [12] GHz
Selected most central building: 37 (distance from origin: 34.02m)
gNB placed on building 37 at position: [-23.77073860168457, -24.334800720214844, 15.5]

SEARCHING FOR VALID ZONE
Distance range: 50m - 300m from TX
Max attempts: 200

Attempt 1/200:
  Zone placed at 172.9m, 243.8° from TX

ZONE VALIDATION
Tx Position: [-23.77073860168457, -24.334800720214844, 15.5]
look_at_position: [-100.06730651855469, -179.5360565185547, 1.5]
TX Position: (-23.8, -24.3, 15.5)

/home/tingjunlab/anaconda3/lib/python3.13/site-packages/scipy/stats/_qmc.py:993: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


  [DEBUG] PathSolver found 50 paths for 50 receivers
  [DEBUG] Total path power (linear): 6.58168e-07
  Gradients: dAz=+8.589e-02°, dEl=-8.332e-03° (norm=8.630e-02)
  Loss: 13.3024, Mean Power in Zone: -13.30 dBm

  [WARNING] PathSolver found 0 paths! (Empty count: 1/5)
            Azimuth=286.6°, Elevation=-5.0°
            Injecting synthetic gradient to guide optimizer.

  Gradients: dAz=-1.000e-02°, dEl=-1.000e-01° (norm=1.005e-01)
  Loss: -10000000000.0000, Mean Power in Zone: 10000000000.00 dBm

  [WARNING] PathSolver found 0 paths! (Empty count: 2/5)
            Azimuth=285.5°, Elevation=-3.5°
            Injecting synthetic gradient to guide optimizer.

  Gradients: dAz=-1.000e-02°, dEl=-1.000e-01° (norm=1.005e-01)
  Loss: -10000000000.0000, Mean Power in Zone: 10000000000.00 dBm

  [WARNING] PathSolver found 0 paths! (Empty count: 3/5)
            Azimuth=284.7°, Elevation=-2.0°
            Injecting synthetic gradient to guide optimizer.

  Gradients: dAz=-1.000e-02°, dEl=-1.

KeyboardInterrupt: 

# Post-processing analysis: Compare across TEST MATRIX combinations
# Analyze performance by Scene, Frequency, Sampler, and LDS method

import pandas as pd
from itertools import product

print("="*80)
print("TEST MATRIX ANALYSIS")
print("="*80)

# ===== Parse Results Keys to Extract Dimensions =====
# Results are keyed as: scene_name + sampler + freq + lds

# Define the test matrix dimensions
SAMPLERS = ['Rejection', 'CDT']
FREQUENCIES = [2.4e9, 5.2e9, 7.1e9, 9.7e9]
FREQ_LABELS = ['2.4 GHz', '5.2 GHz', '7.1 GHz', '9.7 GHz']
LDS_METHODS = ['Rejection', 'Sobol', 'Halton', 'Latin']

# Valid combinations based on the filtering logic
VALID_COMBINATIONS = [
    ('Rejection', 'Rejection'),  # Rejection sampler only uses Rejection LDS
    ('CDT', 'Sobol'),
    ('CDT', 'Halton'),
    ('CDT', 'Latin'),
]

# Parse results into structured format
parsed_results = []

for key, data in results.items():
    if data.get('status') != 'success':
        continue
    
    # Parse the key to extract components
    # Key format: scene_name + sampler + freq + lds
    for sampler, lds in VALID_COMBINATIONS:
        for freq, freq_label in zip(FREQUENCIES, FREQ_LABELS):
            freq_str = f"{freq}"
            test_suffix = sampler + freq_str + lds
            
            if key.endswith(test_suffix):
                scene_name = key[:-len(test_suffix)]
                
                # Calculate improvement metrics
                zone_power_initial = data['zone_power_initial']
                zone_power_optimized = data['zone_power_optimized']
                
                mean_improvement = np.mean(zone_power_optimized) - np.mean(zone_power_initial)
                median_improvement = np.median(zone_power_optimized) - np.median(zone_power_initial)
                p10_improvement = np.percentile(zone_power_optimized, 10) - np.percentile(zone_power_initial, 10)
                p90_improvement = np.percentile(zone_power_optimized, 90) - np.percentile(zone_power_initial, 90)
                
                parsed_results.append({
                    'key': key,
                    'scene': scene_name,
                    'sampler': sampler,
                    'frequency': freq,
                    'freq_label': freq_label,
                    'lds': lds,
                    'combination': f"{sampler}/{lds}",
                    'mean_initial_dBm': np.mean(zone_power_initial),
                    'mean_optimized_dBm': np.mean(zone_power_optimized),
                    'median_initial_dBm': np.median(zone_power_initial),
                    'median_optimized_dBm': np.median(zone_power_optimized),
                    'mean_improvement_dB': mean_improvement,
                    'median_improvement_dB': median_improvement,
                    'p10_improvement_dB': p10_improvement,
                    'p90_improvement_dB': p90_improvement,
                    'final_loss': data['loss_hist'][-1] if data.get('loss_hist') else None,
                })
                break

# Convert to DataFrame
df = pd.DataFrame(parsed_results)

if len(df) == 0:
    print("No successful results to analyze!")
    print("="*80)
else:
    print(f"\nParsed {len(df)} successful experiment results")
    print(f"Scenes: {df['scene'].nunique()}")
    print(f"Frequencies: {df['freq_label'].nunique()}")
    print(f"Combinations: {df['combination'].nunique()}")
    
    # ===== 1. SUMMARY TABLE: Best Combination per Scene =====
    print("\n" + "="*80)
    print("BEST COMBINATION PER SCENE (by mean improvement)")
    print("="*80)
    
    best_per_scene = df.loc[df.groupby('scene')['mean_improvement_dB'].idxmax()]
    print(f"\n{'Scene':<25} {'Best Combination':<20} {'Frequency':<12} {'Improvement':>12}")
    print("-"*70)
    for _, row in best_per_scene.iterrows():
        print(f"{row['scene']:<25} {row['combination']:<20} {row['freq_label']:<12} {row['mean_improvement_dB']:>+10.2f} dB")
    
    # ===== 2. SUMMARY TABLE: Best Combination per Frequency =====
    print("\n" + "="*80)
    print("BEST COMBINATION PER FREQUENCY (averaged across scenes)")
    print("="*80)
    
    freq_combo_avg = df.groupby(['freq_label', 'combination'])['mean_improvement_dB'].mean().reset_index()
    best_per_freq = freq_combo_avg.loc[freq_combo_avg.groupby('freq_label')['mean_improvement_dB'].idxmax()]
    
    print(f"\n{'Frequency':<12} {'Best Combination':<20} {'Avg Improvement':>15}")
    print("-"*50)
    for _, row in best_per_freq.iterrows():
        print(f"{row['freq_label']:<12} {row['combination']:<20} {row['mean_improvement_dB']:>+13.2f} dB")
    
    # ===== 3. HEATMAP: Mean Improvement by Combination and Frequency =====
    print("\n" + "="*80)
    print("PERFORMANCE MATRIX: Mean Improvement (dB) by Combination × Frequency")
    print("="*80)
    
    # Create pivot table
    pivot_table = df.pivot_table(
        values='mean_improvement_dB',
        index='combination',
        columns='freq_label',
        aggfunc='mean'
    )
    
    # Reorder columns by frequency
    freq_order = ['2.4 GHz', '5.2 GHz', '7.1 GHz', '9.7 GHz']
    pivot_table = pivot_table.reindex(columns=[f for f in freq_order if f in pivot_table.columns])
    
    # Print as formatted table
    print(f"\n{'Combination':<20}", end='')
    for col in pivot_table.columns:
        print(f"{col:>12}", end='')
    print(f"{'Mean':>12}")
    print("-" * (20 + 12 * (len(pivot_table.columns) + 1)))
    
    for idx, row in pivot_table.iterrows():
        print(f"{idx:<20}", end='')
        for val in row:
            if pd.notna(val):
                print(f"{val:>+10.2f} dB", end='')
            else:
                print(f"{'N/A':>12}", end='')
        row_mean = row.mean()
        print(f"{row_mean:>+10.2f} dB")
    
    # Column means
    print("-" * (20 + 12 * (len(pivot_table.columns) + 1)))
    print(f"{'Mean':<20}", end='')
    for col in pivot_table.columns:
        col_mean = pivot_table[col].mean()
        print(f"{col_mean:>+10.2f} dB", end='')
    print(f"{pivot_table.values.mean():>+10.2f} dB")
    
    # ===== 4. DETAILED SCENE × FREQUENCY × COMBINATION ANALYSIS =====
    print("\n" + "="*80)
    print("DETAILED RESULTS BY SCENE")
    print("="*80)
    
    for scene in sorted(df['scene'].unique()):
        scene_df = df[df['scene'] == scene]
        print(f"\n▶ {scene}")
        print(f"  {'Frequency':<12} {'Combination':<20} {'Initial':>10} {'Optimized':>10} {'Δ Mean':>10} {'Δ Median':>10}")
        print("  " + "-"*75)
        
        for _, row in scene_df.sort_values(['frequency', 'combination']).iterrows():
            print(f"  {row['freq_label']:<12} {row['combination']:<20} "
                  f"{row['mean_initial_dBm']:>8.1f} dBm {row['mean_optimized_dBm']:>8.1f} dBm "
                  f"{row['mean_improvement_dB']:>+8.2f} dB {row['median_improvement_dB']:>+8.2f} dB")
        
        # Best for this scene
        best_row = scene_df.loc[scene_df['mean_improvement_dB'].idxmax()]
        print(f"  ★ Best: {best_row['combination']} @ {best_row['freq_label']} ({best_row['mean_improvement_dB']:+.2f} dB)")
    
    # ===== 5. VISUALIZATION =====
    print("\n" + "="*80)
    print("GENERATING VISUALIZATIONS")
    print("="*80)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # --- Plot 1: Heatmap of Mean Improvement ---
    ax1 = axes[0, 0]
    heatmap_data = pivot_table.values
    im = ax1.imshow(heatmap_data, cmap='RdYlGn', aspect='auto')
    ax1.set_xticks(range(len(pivot_table.columns)))
    ax1.set_xticklabels(pivot_table.columns, fontsize=10)
    ax1.set_yticks(range(len(pivot_table.index)))
    ax1.set_yticklabels(pivot_table.index, fontsize=10)
    ax1.set_title('Mean Improvement (dB) by Combination × Frequency', fontsize=12, fontweight='bold')
    
    # Annotate heatmap
    for i in range(len(pivot_table.index)):
        for j in range(len(pivot_table.columns)):
            val = heatmap_data[i, j]
            if pd.notna(val):
                color = 'white' if abs(val) > (heatmap_data.max() - heatmap_data.min()) / 2 else 'black'
                ax1.text(j, i, f'{val:+.1f}', ha='center', va='center', fontsize=9, color=color)
    
    plt.colorbar(im, ax=ax1, label='Improvement (dB)')
    
    # --- Plot 2: Bar Chart Comparing Combinations ---
    ax2 = axes[0, 1]
    combo_means = df.groupby('combination')['mean_improvement_dB'].agg(['mean', 'std']).reset_index()
    combo_means = combo_means.sort_values('mean', ascending=False)
    
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(combo_means)))
    bars = ax2.bar(range(len(combo_means)), combo_means['mean'], 
                   yerr=combo_means['std'], capsize=5, color=colors, edgecolor='black')
    ax2.set_xticks(range(len(combo_means)))
    ax2.set_xticklabels(combo_means['combination'], rotation=45, ha='right', fontsize=10)
    ax2.set_ylabel('Mean Improvement (dB)', fontsize=11)
    ax2.set_title('Average Improvement by Combination\n(error bars = std dev across scenes/freqs)', fontsize=12, fontweight='bold')
    ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax2.grid(True, alpha=0.3, axis='y')
    
    # --- Plot 3: Improvement by Frequency ---
    ax3 = axes[1, 0]
    freq_order = ['2.4 GHz', '5.2 GHz', '7.1 GHz', '9.7 GHz']
    combinations = df['combination'].unique()
    x = np.arange(len([f for f in freq_order if f in df['freq_label'].unique()]))
    width = 0.8 / len(combinations)
    
    for i, combo in enumerate(sorted(combinations)):
        combo_data = df[df['combination'] == combo].groupby('freq_label')['mean_improvement_dB'].mean()
        combo_data = combo_data.reindex([f for f in freq_order if f in combo_data.index])
        ax3.bar(x + i * width, combo_data.values, width, label=combo, alpha=0.8)
    
    ax3.set_xticks(x + width * (len(combinations) - 1) / 2)
    ax3.set_xticklabels([f for f in freq_order if f in df['freq_label'].unique()])
    ax3.set_xlabel('Frequency', fontsize=11)
    ax3.set_ylabel('Mean Improvement (dB)', fontsize=11)
    ax3.set_title('Improvement by Frequency and Combination', fontsize=12, fontweight='bold')
    ax3.legend(loc='best', fontsize=9)
    ax3.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax3.grid(True, alpha=0.3, axis='y')
    
    # --- Plot 4: Box Plot by Combination ---
    ax4 = axes[1, 1]
    combo_order = combo_means['combination'].tolist()
    box_data = [df[df['combination'] == c]['mean_improvement_dB'].values for c in combo_order]
    bp = ax4.boxplot(box_data, labels=combo_order, patch_artist=True)
    
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax4.set_xticklabels(combo_order, rotation=45, ha='right', fontsize=10)
    ax4.set_ylabel('Mean Improvement (dB)', fontsize=11)
    ax4.set_title('Distribution of Improvements by Combination', fontsize=12, fontweight='bold')
    ax4.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax4.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # ===== 6. STATISTICAL SUMMARY =====
    print("\n" + "="*80)
    print("STATISTICAL SUMMARY")
    print("="*80)
    
    print("\nOverall Best Combination:")
    overall_best = df.groupby('combination')['mean_improvement_dB'].mean().idxmax()
    overall_best_val = df.groupby('combination')['mean_improvement_dB'].mean().max()
    print(f"  ★ {overall_best}: {overall_best_val:+.2f} dB average improvement")
    
    print("\nBest Combination per Frequency:")
    for freq in [f for f in freq_order if f in df['freq_label'].unique()]:
        freq_df = df[df['freq_label'] == freq]
        best_combo = freq_df.groupby('combination')['mean_improvement_dB'].mean().idxmax()
        best_val = freq_df.groupby('combination')['mean_improvement_dB'].mean().max()
        print(f"  {freq}: {best_combo} ({best_val:+.2f} dB)")
    
    print("\nWin Count by Combination (best per scene×frequency):")
    df['is_best'] = df.groupby(['scene', 'freq_label'])['mean_improvement_dB'].transform(lambda x: x == x.max())
    win_counts = df[df['is_best']].groupby('combination').size()
    total_experiments = df.groupby(['scene', 'freq_label']).ngroups
    for combo, count in win_counts.sort_values(ascending=False).items():
        print(f"  {combo}: {count}/{total_experiments} wins ({100*count/total_experiments:.1f}%)")
    
    print("\n" + "="*80)
    print("Analysis complete!")
    print("="*80)